Используем также трансформер с huggingface

In [1]:
import pandas as pd, numpy as np 
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification
import tokenizers
print('TF version', tf.__version__)
print(tf.test.is_gpu_available())

2025-04-16 18:48:37.297256: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 18:48:37.453423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744818517.523407   36547 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744818517.544896   36547 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744818517.676525   36547 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

TF version 2.19.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


I0000 00:00:1744818524.507877   36547 gpu_device.cc:2019] Created device /device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Подготовка данных

Для моей 3050

In [3]:
import kagglehub

path = kagglehub.dataset_download("mdismielhossenabir/sentiment-analysis")

print("Path to dataset files:", path)

Path to dataset files: /home/t33nsy/.cache/kagglehub/datasets/mdismielhossenabir/sentiment-analysis/versions/1


In [5]:
df = pd.read_csv(path+"/sentiment_analysis.csv", header=None)
df.head(5)

,0,1,2,3,4,5,6
0,Year,Month,Day,Time of Tweet,text,sentiment,Platform
1,2018,8,18,morning,What a great day!!! Looks like dream.,positive,Twitter
2,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",positive,Facebook
3,2017,8,18,night,Don't angry me,negative,Facebook
4,2022,6,8,morning,We attend in the class just for listening teac...,negative,Facebook


In [10]:
df.columns = ["year", "month", "day", "time", "content", "sentiment", "platform"]
df = df.iloc[1:]
df = df.dropna(subset=["content", "sentiment"])
df["content"] = df["content"].astype(str)
df["sentiment"] = df["sentiment"].astype(str)
df.head(5)

,year,month,day,time,content,sentiment,platform
1,2018,8,18,morning,What a great day!!! Looks like dream.,positive,Twitter
2,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",positive,Facebook
3,2017,8,18,night,Don't angry me,negative,Facebook
4,2022,6,8,morning,We attend in the class just for listening teac...,negative,Facebook
5,2022,6,8,noon,"Those who want to go, let them go",negative,Instagram


In [11]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["sentiment"])
df.head(5)

,year,month,day,time,content,sentiment,platform,label
1,2018,8,18,morning,What a great day!!! Looks like dream.,positive,Twitter,2
2,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",positive,Facebook,2
3,2017,8,18,night,Don't angry me,negative,Facebook,0
4,2022,6,8,morning,We attend in the class just for listening teac...,negative,Facebook,0
5,2022,6,8,noon,"Those who want to go, let them go",negative,Instagram,0


In [12]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["content"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

In [13]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

In [ ]:
max_len = 96
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding='max_length',
    max_length=max_len,
    return_tensors="tf"
)
val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding='max_length',
    max_length=max_len,
    return_tensors="tf"
)

I0000 00:00:1744818663.602019   36547 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [15]:
labels_num = df["sentiment"].nunique()
print(labels_num)
print(df["sentiment"].value_counts())

3
sentiment
neutral     199
positive    166
negative    134
Name: count, dtype: int64


In [16]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(4)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(4)

In [17]:
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=labels_num)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [18]:
dummy_input = tf.constant([[0]*96])  # фиктивный input_id (нужно под размер токена)
model(dummy_input)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 0.20234898, -0.12209343,  0.02125356]], dtype=float32)>, hidden_states=None, attentions=None)

In [19]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  124055040 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  592899    
 ificationHead)                                                  
                                                                 
Total params: 124647939 (475.49 MB)
Trainable params: 124647939 (475.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Создаем свои коллбэки, так как керас конфликтит

In [ ]:
epochs = 5
for i in range(epochs):
    model.fit(train_dataset, validation_data=val_dataset, epochs=1)
    model.save_weights(f"roberta_epoch_{3:02d}.weights.h5")

Epoch 1/3


2025-04-16 18:52:35.455671: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.50MiB (rounded to 4718592)requested by op tf_roberta_for_sequence_classification/roberta/encoder/layer_._9/intermediate/Gelu/Erfc
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-04-16 18:52:35.455746: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-04-16 18:52:35.455756: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 117, Chunks in use: 116. 29.2KiB allocated for chunks. 29.0KiB in use in bin. 789B client-requested in use in bin.
2025-04-16 18:52:35.455758: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated for chunk

ResourceExhaustedError: Graph execution error:

Detected at node tf_roberta_for_sequence_classification/roberta/encoder/layer_._9/intermediate/Gelu/Erfc defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/t33nsy/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/t33nsy/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/t33nsy/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/t33nsy/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/t33ns